In [ ]:
import pandas as pd 
import numpy as np 
import glob
import torch

In [ ]:
data = {
    "brain": [], 
    "butterfly": [],
}



In [ ]:
for i in glob.glob("caltech-101/101_ObjectCategories/brain/*"):
    data["brain"].append(i)
data

In [ ]:
for i in glob.glob("caltech-101/101_ObjectCategories/butterfly/*"):
    data["butterfly"].append(i)

In [ ]:
data["brain"] = data["brain"][:10]
data["butterfly"] = data["butterfly"][:10]


In [ ]:
embedding_data = []
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn 
model = resnet50( weights=ResNet50_Weights.IMAGENET1K_V2)

model = nn.Sequential(*list(model.children())[:-1])


In [ ]:
import cv2
for brain_path in data["brain"]: 
    img = cv2.imread(brain_path)
    img = cv2.resize(img, (224,224))
    

    transposed_img = img.transpose(2, 0, 1)
    features = model(torch.Tensor([transposed_img]))
    flattened_features = features.view(features.size(0), -1)[0].tolist()

    embedding_data.append(flattened_features)

    


In [ ]:
for bf_path in data["butterfly"]: 
    img = cv2.imread(bf_path)
    img = cv2.resize(img, (224,224))
    

    transposed_img = img.transpose(2, 0, 1)
    features = model(torch.Tensor([transposed_img]))
    flattened_features = features.view(features.size(0), -1)[0].tolist()
    embedding_data.append(flattened_features)

    

In [ ]:
print(data["brain"])

In [ ]:
import chromadb
client = chromadb.Client()
collection = client.get_or_create_collection("image-searches")

In [ ]:
collection.add(embeddings=embedding_data, ids=[f"id-{x}" for x in range(len(embedding_data))])


In [ ]:
img = cv2.imread("caltech-101/101_ObjectCategories/butterfly/image_0041.jpg")
img = cv2.resize(img, (224,224)).transpose(2,0,1)
features = model(torch.Tensor([img]))
flattened_features = features.view(features.size(0), -1)[0].tolist()

In [ ]:
img = cv2.imread("caltech-101/101_ObjectCategories/brain/image_0033.jpg")
img = cv2.resize(img, (224,224)).transpose(2,0,1)
features = model(torch.Tensor([img]))
flattened_features = features.view(features.size(0), -1)[0].tolist()

In [ ]:
results = collection.query(query_embeddings=flattened_features, n_results=2, include=["embeddings"])